In [2]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language
from typing import List
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from sentence_transformers import SentenceTransformer
from LocalEmbedding import LocalEmbeddings
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import logging
import random
import math

def get_env_variable(var_name: str) -> str:
    """
    获取环境变量的值，如果环境变量不存在则抛出异常

    Args:
    var_name (str): 环境变量名称

    Returns:
    str: 环境变量的值

    Raises:
    ValueError: 如果环境变量未设置
    """
    value = os.getenv(var_name)
    if value is None:
        raise ValueError(f"环境变量 '{var_name}' 未设置")
    return value


# 加载.env文件中的环境变量
load_dotenv() 

d:\Anaconda\envs\RAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
# 加载文档
# repo_path = get_env_variable("Repo_Path")+ "/libs/core/langchain_core" # 测试用的
repo_path = get_env_variable("Python_Path")

# 获取所有子文件夹
subfolders = [os.path.join(repo_path, f) for f in os.listdir(repo_path) if os.path.isdir(os.path.join(repo_path, f))]
total_subfolders = len(subfolders)
print(f"总共有 {total_subfolders} 个子文件夹。")

# 计算需要随机选取的文件夹数量，向上取整
num_to_select = math.ceil(total_subfolders * 0.4)
selected_subfolders = random.sample(subfolders, num_to_select)
print(f"随机选取了 {num_to_select} 个子文件夹进行读取。")

# 创建 GenericLoader 实例，并显示进度条
for folder in selected_subfolders:
    print(f"正在处理文件夹: {folder}")
    loader = GenericLoader.from_filesystem(
        path=folder,
        glob="**/*",
        suffixes=[".py"],
        exclude=["**/non-utf8-encoding.py"],
        parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),  # parser_threshold: Minimum lines needed to activate parsing (0 by default).
        show_progress=True  # 显示进度条
    )
    documents = list(loader.lazy_load())
    print(f"文件夹 {folder} 中加载了 {len(documents)} 个文档。")

# loader = GenericLoader.from_filesystem(
#     path = repo_path,
#     glob="**/*",
#     suffixes=[".py"],
#     exclude=["**/non-utf8-encoding.py"],
#     parser=LanguageParser(language=Language.PYTHON, parser_threshold=500), # parser_threshold: Minimum lines needed to activate parsing (0 by default).
#     show_progress=True  # 显示进度条
# )

总共有 800 个子文件夹。
随机选取了 320 个子文件夹进行读取。
正在处理文件夹: D:/Project/data/Project_CodeNet_Python800\p04033


100%|██████████| 300/300 [00:00<00:00, 1300.47it/s]


文件夹 D:/Project/data/Project_CodeNet_Python800\p04033 中加载了 300 个文档。
正在处理文件夹: D:/Project/data/Project_CodeNet_Python800\p02842


100%|██████████| 300/300 [00:00<00:00, 3868.67it/s]


文件夹 D:/Project/data/Project_CodeNet_Python800\p02842 中加载了 300 个文档。
正在处理文件夹: D:/Project/data/Project_CodeNet_Python800\p00007


...

文件夹 D:/Project/data/Project_CodeNet_Python800\p03337 中加载了 300 个文档。
正在处理文件夹: D:/Project/data/Project_CodeNet_Python800\p02388


100%|██████████| 300/300 [00:00<00:00, 4109.74it/s]


文件夹 D:/Project/data/Project_CodeNet_Python800\p02388 中加载了 300 个文档。
正在处理文件夹: D:/Project/data/Project_CodeNet_Python800\p03150


100%|██████████| 300/300 [00:00<00:00, 3797.79it/s]

文件夹 D:/Project/data/Project_CodeNet_Python800\p03150 中加载了 300 个文档。


In [4]:
# 获取文档生成器并展开
document_list = list(loader.lazy_load())
print(f"len(document_list): {len(document_list)}")


# 并行加载文档
documents = []

# 打印 CPU 核心数
cpu_count = os.cpu_count()
print(f"系统中可用的 CPU 核心数: {cpu_count}")
with ThreadPoolExecutor(max_workers=cpu_count) as executor:
    futures = [executor.submit(lambda doc: doc, doc) for doc in document_list]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Loading documents"):
        documents.append(future.result())

100%|██████████| 300/300 [00:00<00:00, 3881.06it/s]


len(document_list): 300
系统中可用的 CPU 核心数: 12


Loading documents: 100%|██████████| 300/300 [00:00<00:00, 150207.85it/s]


In [5]:
# Splitter
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
print(len(texts))

303


In [6]:
# RetrievalQA
model_path = get_env_variable("Local_Model_acge_text_embedding")
# 初始化 LocalEmbeddings 实例，使用本地模型的路径
local_embeddings = LocalEmbeddings(
    model_path=model_path,  # 本地模型文件的路径
)
# 初始化 LocalEmbeddings 实例，使用本地模型和分词器的路径
# model_path = get_env_variable("Local_Model_acge_text_embedding")
# tokenizer_path = get_env_variable("Local_Tokenizer_Path")
# local_embeddings = LocalEmbeddings(
#     model_path=model_path,  # 本地模型文件的路径
#     tokenizer_path=tokenizer_path  # 本地分词器的路径
# )

In [7]:
# 初始化一个空的 Chroma 数据库
final_db = Chroma(embedding_function=local_embeddings)

# 定义批量计算嵌入的函数
def compute_embeddings(batch_texts, batch_index, total_batches, progress_bar):
    logging.debug(f"开始处理批次 {batch_index + 1}/{total_batches}: {len(batch_texts)} 个文档")
    try:
        # 使用 add_documents 方法将文档添加到 Chroma 数据库中
        final_db.add_documents(batch_texts)
        logging.debug(f"处理完成批次 {batch_index + 1}/{total_batches}: {len(batch_texts)} 个文档")
    except Exception as e:
        logging.error(f"处理批次 {batch_index + 1}/{total_batches} 时出错: {e}")
    finally:
        progress_bar.update(1)

# 设置批处理大小
batch_size = 100  # 批处理大小
batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
total_batches = len(batches)
print(f"总批次{total_batches}")

# 并行计算嵌入并合并结果
with ThreadPoolExecutor(max_workers=cpu_count) as executor, tqdm(total=total_batches, desc="Computing embeddings") as progress_bar:
    futures = [executor.submit(compute_embeddings, batch, i, total_batches, progress_bar) for i, batch in enumerate(batches)]
    for future in as_completed(futures):
        future.result()

print("Document database created, final_db=", final_db)

总批次4


Computing embeddings: 100%|██████████| 4/4 [00:11<00:00,  2.93s/it]

Document database created, final_db= <langchain_chroma.vectorstores.Chroma object at 0x0000025783E54130>


In [9]:
# 创建检索器，可以使用 'mmr' 或 'similarity' 检索类型
retriever = final_db.as_retriever(
    search_type="mmr",  # 或 "similarity"
    search_kwargs={"k": 8}  # k 表示检索的文档数量
)

In [10]:
# 调用chatglm
# chatglm_api_key = get_env_variable("ChatGLM_API_KEY_glm4")
# chatglm_api_base = get_env_variable("ChatGLM_API_BASE")
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(
#     temperature=0.95,
#     model="glm-4",
#     openai_api_key=chatglm_api_key,
#     openai_api_base=chatglm_api_base
# )
# 调用本地ollama
llm = ChatOllama(
    model="llama3",
    temperature=0.9,
    # other params...
)

In [ ]:
# First we need a prompt that we can pass into an LLM to generate this search query
import requests
prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)
document_chain = create_stuff_documents_chain(llm, prompt)

qa = create_retrieval_chain(retriever_chain, document_chain)

questions = [
    "What is a RunnableBinding?",
    "What classes are derived from the Runnable class?",
    "What one improvement do you propose in code in relation to the class hierarchy for the Runnable class?",
]

# 示例调用方法
def invoke_question(question):
    try:
        result = qa.invoke({"input": question})  # 这里假设 qa 是一个已经定义的对象
        return result['answer']
    except requests.exceptions.RequestException as e:
        print(f"请求错误: {e}")
        if e.response is not None:
            print(f"响应内容: {e.response.text}")
            print(f"状态码: {e.response.status_code}")
        return None

for question in questions:
    result = invoke_question({"input": question})
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")